In [ ]:
using DataFrames, CSV, MLJ, MLJLinearModels, DecisionTree, VegaLite

In [ ]:
# Import training dataset
lichen_training = CSV.read("C:/Users/jnovoa/Downloads/lichen_training.csv", DataFrame) |>
x -> select(x, "Total.lichen", "basal_area", "PL", "mean_Long") |>
x -> rename(x, ["biomass", "basalarea", "pl", "lon"]) |>
x -> coerce(x, Count => Continuous);

In [ ]:
# Calculate log(biomass) to avoid predicted negative values
lichen_training[!, "biomass_log"] = log.(lichen_training[!, :biomass])
select!(lichen_training, Not(:biomass))
schema(lichen_training)

In [ ]:
# Split predictors (X) and response variable (y)
y, X = unpack(lichen_training, ==(:biomass_log), _ -> true);

In [ ]:
# Load a linear regressor and a random forest regressor
LinearRegressor = @load LinearRegressor pkg=MLJLinearModels
RFRegressor = @load RandomForestRegressor pkg=DecisionTree

In [ ]:
# Instantiate a pipeline
RegressorPipe = @pipeline(Standardizer(), LinearRegressor(), RFRegressor())